In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import time




In [ ]:
#geospatial map imports
from shapely.geometry import Point
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from folium import IFrame

# Static Setup (Not Redefined in loop)
- Fire Stations DF
- Zip Codes DF
- geostations
- remember that "zip" will be the standard column names for 

#### Station Count

In [ ]:
## Bring in FireStations Data
endpoint = "https://data.nashville.gov/resource/frq9-a5iv.json" #fire stations endpoint

response = requests.get(endpoint)
stations = response.json()
stations_df = pd.DataFrame(stations)
delcol = (['mapped_location', ':@computed_region_gisn_y5cm', 
           ':@computed_region_sjpq_96s8',':@computed_region_v3ji_vzam',
           ':@computed_region_c9xn_skx3',':@computed_region_cfa7_hbpz',
            ':@computed_region_f73m_vb2k', 'street_address','city','state'])



In [ ]:
station_count = stations_df.groupby('zip_code').count().drop(columns = delcol)
station_count = station_count.rename_axis('zip_code').reset_index()
station_count = station_count.rename(columns = {'station_number':'num_stations','zip_code':'zip'})

#### zipcodes

In [ ]:
zipcodes = gpd.read_file('C:/Users/theoc/Documents/NSS/Python/data/zipcodes.geojson')

#### Geostations

In [ ]:
geostations = pd.DataFrame(stations)

delcol = ([':@computed_region_gisn_y5cm', 
           ':@computed_region_sjpq_96s8',':@computed_region_v3ji_vzam',
           ':@computed_region_c9xn_skx3',':@computed_region_cfa7_hbpz',
            ':@computed_region_f73m_vb2k', 'street_address','city','state'])
geostations = geostations.drop(columns=delcol)
geostations = geostations.rename(columns = {'zip_code':'zip'})

#this is a really important step so when we rerun the notebook the station with null geography can
#be dropped
geostations = geostations.sort_values(by = 'zip').reset_index(drop=True)
#perform the drop
geostations = geostations.drop(37)
#reset index last time
geostations = geostations.reset_index(drop=True)

In [ ]:
station_list = []
zip_list = []
long_list = []
lat_list = []
for index, row in geostations.iterrows():
    long_list.append(row['mapped_location'].get('longitude'))
    lat_list.append(row['mapped_location'].get('latitude'))
    station_list.append(row['station_number'])
    zip_list.append(row['zip'])
geostations_clean = pd.DataFrame({'station_number':station_list,
                                  'zip':zip_list,
                                  'long':long_list,
                                  'lat':lat_list})
geostations_clean.head()

# Auto Refresh Loop

In [ ]:
from datetime import datetime as dt

In [ ]:
photo_path = (
"C:/Users/theoc/Documents/NSS/Python/projects/socrata_api-theo-carr/notebooks/vault_maps/"
)

In [ ]:
## already have zipcodes, geostations, and firestations
## we need a running dataframe to keep all our results that tracks
## add our workload dataframe onto a big running one and collect a large dataset
the_vault = pd.DataFrame()
t = 0 #iterations

while t < 150:
    #pull requests from incidents
    #join incidents with station count
    #make map
    #save map and add to dataframe
    #sleep for 5.5 minutes
    
    
    ##############################################################
    ### GET ACTIVE INCIDENTS ### 
    endpoint = "https://data.nashville.gov/resource/jwgg-8gg4.json"

    response = requests.get(endpoint)
    incidents = response.json()
    incidents_df = pd.DataFrame(incidents)

    incident_counts = incidents_df.groupby('postal_code').count().drop(columns = 
                                                ['dispatch_time','incident_type','units_dispatched'])
    incident_counts = incident_counts.rename(columns={'incident_number':'num_incidents'})

    incident_counts = incident_counts.rename_axis('zip_code').reset_index()
    
    incident_counts = incident_counts.rename(columns = {'zip_code':'zip'})
    #######################################################################
    ### MERGE WITH STATION COUNTS(STATIC) ### 
    station_workload = pd.merge(station_count, incident_counts, how='left', on='zip')
    station_workload['incidents_to_stations_ratio'] = (station_workload['num_incidents'] / 
                                                       station_workload['num_stations'])
    station_workload = station_workload.fillna(0)
    ########################################################################
    #MAKE ZIPMAP DF 
    zipmap = pd.merge(zipcodes, station_workload, how='left', left_on = 'zip', right_on = 'zip')
    zipmap = zipmap.fillna(0)
    ########################################################################
    #Make the plot map since we can save it as an image file instead of html
    #make the plot using matplot lib
    #make the plot using matplot lib
    fig,ax = plt.subplots(figsize=(50,30))
    zipmap.plot(column = 'incidents_to_stations_ratio', ax=ax, edgecolor='black',
                cmap = 'Reds', vmin = 0, vmax = 5,
                legend=True)
    ax.axis('off')
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(-.05, 0.95, time.ctime(), transform=ax.transAxes, fontsize=50,
            verticalalignment='top', bbox=props)
    filepath='C:/Users/theoc/Documents/NSS/Python/projects/socrata_api-theo-carr/notebooks/imgs/night run'
    plt.savefig(filepath+'test'+str(t))
    plt.close()


    ##########################################################################
    #MAKE THE MAP!!!! 
    #get curr time
    timestamp = dt.now()
    #base map = nashmap
    nashmap = folium.Map(location = [36.148385041463946,-86.77666862070282],zoom_start=10,tiles='stamentoner')
    #add time stamp
    #!!!not figured out yet!!!#
    
    
    #draw the zip boundries
    folium.Choropleth(geo_data = zipmap, 
                      data = zipmap, 
                      columns=['zip','incidents_to_stations_ratio'],
                      key_on="feature.properties.zip",
                      fill_color='Reds',
                      nan_fill_color="White", #Use white color if there is no data available for the county
                      fill_opacity=0.8,
                      line_opacity=1,
                      legend_name='Station Strain', #title of the legend
                      highlight=True,
                      line_color='black'
                      ).add_to(nashmap)
    # Add fire station markers
    for index, row in geostations_clean.iterrows():
        loc = [row['lat'], row['long']]
        pop = str(row['station_number'])

       # icon = folium.Icon(color='darkred',icon = 'fire-extinguisher', prefix = 'fa',edgecolor='black')
        marker = folium.Circle(location = loc, 
                               tooltip = pop,
                               radius = 350,
                               color='black', fill=True, fill_opacity=.6,opacity=.1, 
                               size=(10,10))
        marker.add_to(nashmap)

    #save map
    timestamp = timestamp.strftime("%b-%Hhr-%Mmin")
    ###nashmap.save(photo_path+timestamp+'.html')
    ##not saving map for overnight, too memory intensive
    ########################################################################
    #add time stamp column to station workload
    station_workload['timestamp'] = timestamp
    #INCREMENT AND SLEEP! #ADD CURRENT DATA TO LARGER FRAME(THE VAULT) 
    if t == 0: #first iteration
        the_vault = station_workload
    elif t > 0:
        the_vault = pd.concat([the_vault,station_workload])
    t += 1
    #Sleep for 5.5 minutes = 330 Seconds
    time.sleep(320)
    #time.sleep(.5)
#if we some how make it out of the for loop
##prepping the save
csv_path = 'C:/Users/theoc/Documents/NSS/Python/projects/socrata_api-theo-carr/notebooks/cookedData/'
#save
the_vault.to_csv(csv_path+'night_run.csv', index=False)

# ** UNDER NO CIRCUMSTANCES WILL YOU FORGET TO SAVE YOUR VAULT DATA FRAME **

In [ ]:
##prepping the save
csv_path = 'C:/Users/theoc/Documents/NSS/Python/projects/socrata_api-theo-carr/notebooks/cookedData/'
#save
the_vault.to_csv(csv_path+'night_run.csv', index=False)